In [59]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer

# Read in data
df = pd.read_csv('/Users/alexprascak/Downloads/USvideos.csv', names = ['ID', 'Trending_Date', 'title', 'channel_title','category_id','publish_time',
'tags',
'views',
'likes',
'dislikes',
'comment_count',
'thumbnail_link',
'comments_disabled',
'ratings_disabled',
'video_error_or_removed',
'description'])

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,7,8,9,10,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
# Data pre-processing
preprocessed_data = df.drop(['description', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'publish_time', 'channel_title', 'trending_date', 'video_id', 'tags', 'title'], axis=1)
preprocessed_data.head()

,category_id,views,likes,dislikes,comment_count
0,22,748374,57527,2966,15954
1,24,2418783,97185,6146,12703
2,23,3191434,146033,5339,8181
3,24,343168,10172,666,2146
4,24,2095731,132235,1989,17518


In [29]:
# Prepare training and testing data
x = preprocessed_data.drop('category_id', axis = 1)
y = preprocessed_data['category_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [61]:
# Data scaling using min/max
scaling              = MinMaxScaler(feature_range=(0, 1)).fit(x_train)
quantile_scaler      = QuantileTransformer(output_distribution='uniform').fit(x_train)
quantile_transformer = QuantileTransformer(output_distribution='uniform').fit(x_train)

Xq_train  = quantile_scaler.transform(x_train)
Xq_test   = quantile_scaler.transform(x_test)
Xqn_train = quantile_transformer.transform(x_train)
Xqn_test  = quantile_transformer.transform(x_train)

In [ ]:
svc = SVC(kernel='rbf', C=100, decision_function_shape='ovr', gamma = 1000)
svc.fit(Xq_train, y_train)

In [25]:
y_pred = svc.predict(x_test)
accuracy_score(y_test, y_pred)

0.3363858363858364

In [44]:
warnings.simplefilter(action='ignore', category=FutureWarning)
not_null = df[ (df['title'] != '[none]') & (df['tags'] != '[none]') & (df['views'] != '[none]') & (df['likes'] != '[none]') & (df['dislikes'] != '[none]') & (df['comment_count'] != '[none]')]
not_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39415 entries, 0 to 40949
Data columns (total 16 columns):
ID                        39415 non-null object
Trending_Date             39415 non-null object
title                     39415 non-null object
channel_title             39415 non-null object
category_id               39415 non-null object
publish_time              39415 non-null object
tags                      39415 non-null object
views                     39415 non-null object
likes                     39415 non-null object
dislikes                  39415 non-null object
comment_count             39415 non-null object
thumbnail_link            39415 non-null object
comments_disabled         39415 non-null object
ratings_disabled          39415 non-null object
video_error_or_removed    39415 non-null object
description               39120 non-null object
dtypes: object(16)
memory usage: 5.1+ MB


In [45]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Initialize common words from stopwords
common_words = stopwords.words('english')
common_words = common_words + ['']

# Define get_words() function
def get_words(col : str):
    tagsArray = not_null[col]
    tag2D = []
    tagsArray
    
    for tag in tagsArray:
        lineArray = tag.split('|')
        words = []
        for sentence in lineArray:
            words = words + re.split(r'\W+', sentence.strip())
        tag2D.append(words)
        
    for i in range(len(tag2D)):
        tag2D[i] = [u for u in tag2D[i] if u.lower() not in common_words]
        
    return tag2D

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexprascak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
# Function to 
import hashlib
def hashArray(myArray):
    result = []
    for words in myArray:
        hashArray = []
        for word in words:
            hashArray.append(abs(hash(word)))
        result.append(hashArray)
    return result

In [51]:
len(stopwords.words('english'))
wordList = get_words('tags')
hashArray(wordList)

[[3677155665188971302],
 [8698570437422499325, 5573895013677635922],
 [4244313326418051190,
  733616589004611283,
  5993161410630938938,
  6305478520341811056,
  2260157096753823627,
  4244313326418051190,
  733616589004611283,
  5993161410630938938,
  8989262812271518929,
  6305478520341811056,
  6289524699396989372,
  2945829650761029034,
  6305478520341811056,
  8989262812271518929,
  6305478520341811056],
 [9174719297604144626,
  7955421146892185332,
  1079368810427149373,
  8621775600883552747,
  5664739726183281973,
  3952523873520375796,
  9174719297604144626,
  7955421146892185332,
  1024251135609520713,
  1079368810427149373,
  8621775600883552747,
  9029927578181664393,
  2838222586858755906,
  3846185029525993437,
  4668571881223341285,
  7149494137063547564,
  770203577535057914,
  8030122919969364051,
  6190767683075230883,
  1365384523447603071,
  5526311755977650445,
  7954253191334747322,
  1494252418786014204,
  5853956436746155598,
  3211387161863223938,
  19473761901

In [54]:
le = LabelEncoder()
